<a href="https://colab.research.google.com/github/Kumaran-Kugathasan/tamilchatbot/blob/main/Fasttext_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gensim
from gensim.models.fasttext import FastText
import numpy as np
%tensorflow_version 1.x
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint

     |████████████████████████████████| 24.2MB 1.5MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
import pandas as pd
def load_dataset(filename):
  df = pd.read_csv(filename, encoding = "utf-8",
       names = ["Sentence", "Intent"])
  intent = df["Intent"]
  unique_intent = list(set(intent))
  df["Sentence"]=df["Sentence"].apply(str)
  sentences = list(df["Sentence"])
  
  return (intent, unique_intent, sentences)

In [ ]:
intent, unique_intent, sentences = load_dataset("/content/bank.csv")

FileNotFoundError: ignored

In [ ]:
nltk.download("punkt")
def cleaning(sentences):
  words = [] 
  for s in sentences:
    w = word_tokenize(s)
    words.append([i for i in w])     
  return words  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  
print(len(sentences))

1567
[['நான்', '2in1', 'கணக்கில்', 'சேமிப்பது', 'எப்படி', '?'], ['2in1', 'கணக்கில்', 'நான்', 'சேமிப்பது', 'எப்படி', '?']]
1567


In [ ]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
  token = Tokenizer(filters = filters)
  token.fit_on_texts(words)
  return token

In [ ]:
def max_length(words):
  return(len(max(words, key = len)))

In [ ]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 1141 and Maximum length = 22


In [ ]:
print(cleaned_words[0])

['நான்', '2in1', 'கணக்கில்', 'சேமிப்பது', 'எப்படி', '?']


In [ ]:
embedding_size = 300
window_size = 40
min_word = 5
down_sampling = 1e-2

In [ ]:
ft_model = FastText(cleaned_words,
                      size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      sg=1,
                      iter=100)

In [ ]:
print(ft_model.wv['2in1'])
word_tokenizer.word_index.items()

[ 4.62436706e-01  9.17511284e-02 -3.61815423e-01 -4.62018758e-01
  2.19947100e-03 -6.68008387e-01 -2.52920479e-01  8.77033845e-02
  2.07961664e-01  2.90900111e-01 -1.40184462e-01 -9.71336886e-02
 -3.48810583e-01  2.24355027e-01  1.93670049e-01 -2.43046105e-01
 -5.28243138e-04 -2.62768537e-01 -1.12149388e-01  7.21554458e-02
  3.13666791e-01  2.44076639e-01 -3.37892026e-01 -3.04775417e-01
  1.35612730e-02 -5.44601046e-02  1.11329718e-03  3.68816197e-01
  5.34716016e-03  8.39293450e-02 -3.76321226e-01 -1.86926890e-02
 -8.10149461e-02 -3.81423943e-02 -2.17716843e-01 -1.28624424e-01
 -3.04014802e-01  2.24025790e-02 -9.85087380e-02 -9.21506956e-02
 -3.10148835e-01  2.93268293e-01 -3.77210408e-01  3.26381415e-01
 -3.87845635e-01 -5.15883505e-01  5.60604371e-02  1.59502164e-01
  2.96994835e-01 -6.57963902e-02 -2.39603743e-01  2.96860665e-01
  6.01992011e-03 -4.14718121e-01  2.82580912e-01 -2.10049823e-01
 -2.69898355e-01  6.26595989e-02  3.02972317e-01 -3.94741714e-01
  4.12913598e-02  1.57908

dict_items([('?', 1), ('என்ன', 2), ('நான்', 3), ('கடன்', 4), ('எனது', 5), ('boc', 6), ('டிக்கிரி', 7), ('கணக்கை', 8), ('”', 9), ('எவ்வாறு', 10), ('எப்படி', 11), ('அட்டை', 12), ('அன்பளிப்புச்', 13), ('வெளிநாட்டு', 14), ('செலான்', 15), ('முடியும்', 16), ('அட்டையை', 17), ('நாணய', 18), ('“', 19), ('சீட்டு', 20), ('வேண்டும்', 21), ('எவை', 22), ('ஆவணங்கள்', 23), ('பெற', 24), ('.', 25), ('முடியுமா', 26), ('செய்ய', 27), ('தேவையான', 28), ('பெற்றுக்', 29), ('ஆரம்பிக்க', 30), ('fcaispe', 31), ('திறைசேரி', 32), ('சேமிப்பு', 33), ('ஆயுள்', 34), ('எந்த', 35), ('கொள்ள', 36), ('பற்றி', 37), ('எவ்வளவு', 38), ('தொழில்', 39), ('இலங்கை', 40), ('கீழ்', 41), ('நிலையான', 42), ('வழங்குனர்', 43), ('எனக்கு', 44), ('2in1', 45), ('வேண்டிய', 46), ('ஒன்றை', 47), ('புதிய', 48), ('தேவைப்படும்', 49), ('மாதாந்த', 50), ('கணக்கொன்றை', 51), ('கணக்கிற்கான', 52), ('கடனை', 53), ('குறைந்தபட்ச', 54), ('மற்றும்', 55), ('வகையின்', 56), ('வயது', 57), ('தொகை', 58), ('விற்பனை', 59), ('பெறுவது', 60), ('வேண்டுமா', 61), ('கணக்கில்', 6

In [ ]:
word_index =word_tokenizer.word_index

In [ ]:
#embedding matrix

print('preparing embedding matrix...')
words_not_found = []
nb_words = 1141

embedding_matrix = np.zeros((nb_words, embedding_size ))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = ft_model.wv[word]
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

preparing embedding matrix...
number of null word embeddings: 1


In [ ]:
print("sample words not found: ", words_not_found)

sample words not found:  []


In [ ]:
def encoding_doc(token, words):
  return(token.texts_to_sequences(words))

In [ ]:
#tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')
encoded_output = encoding_doc(output_tokenizer, intent)
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)
encoded_output.shape


(1567, 1)

In [ ]:
def one_hot(encode):
  o = OneHotEncoder(sparse = False)
  return(o.fit_transform(encode))

In [ ]:
output_one_hot = one_hot(encoded_output)

In [ ]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [ ]:
def padding_doc(encoded_doc, max_length):
  return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [ ]:
padded_doc = padding_doc(encoded_doc, max_length)

In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (1253, 22) and train_Y = (1253, 56)
Shape of val_X = (314, 22) and val_Y = (314, 56)


In [ ]:
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras import regularizers
from keras import optimizers

batch_size = 256 
num_epochs = 8 

#model parameters
num_filters = 64 
#embed_dim = 300 
weight_decay = 1e-4

num_classes = 56
print("training CNN ...")
model = Sequential()
model.add(Embedding(nb_words, embedding_size, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Conv1D(num_filters, 7, activation='linear', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='linear', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='linear', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(num_classes, activation='sigmoid'))  #multi-label (k-hot encoding)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()


training CNN ...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 22, 300)           342300    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 22, 64)            134464    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 11, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 11, 64)            28736     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
___________________________________________________________

In [ ]:
from keras.callbacks import EarlyStopping
#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

In [ ]:
hist = model.fit(train_X, train_Y, batch_size=batch_size, epochs=num_epochs, callbacks=callbacks_list, validation_split=0.1, shuffle=True, verbose=2)


Train on 1127 samples, validate on 126 samples
Epoch 1/8
 - 1s - loss: 0.6647 - accuracy: 0.6040 - val_loss: 0.5430 - val_accuracy: 0.7823
Epoch 2/8
 - 1s - loss: 0.5534 - accuracy: 0.7355 - val_loss: 0.4029 - val_accuracy: 0.8503
Epoch 3/8
 - 1s - loss: 0.4644 - accuracy: 0.7948 - val_loss: 0.2973 - val_accuracy: 0.9158
Epoch 4/8
 - 1s - loss: 0.3751 - accuracy: 0.8518 - val_loss: 0.2174 - val_accuracy: 0.9520
Epoch 5/8
 - 1s - loss: 0.2849 - accuracy: 0.9006 - val_loss: 0.1558 - val_accuracy: 0.9646
Epoch 6/8
 - 1s - loss: 0.2125 - accuracy: 0.9344 - val_loss: 0.1188 - val_accuracy: 0.9719
Epoch 7/8
 - 1s - loss: 0.1642 - accuracy: 0.9562 - val_loss: 0.1047 - val_accuracy: 0.9820
Epoch 8/8
 - 1s - loss: 0.1360 - accuracy: 0.9707 - val_loss: 0.0964 - val_accuracy: 0.9821


In [ ]:
print(model.test_on_batch(val_X,val_Y))
model.metrics_names

[0.09649597, 0.98214304]


['loss', 'accuracy']

In [ ]:
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
train_X.shape,train_Y.shape


((1253, 22), (1253, 56))

In [ ]:
smote = SMOTE('minority')
X_sm, y_sm = smote.fit_sample(train_X,train_Y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print(X_sm.shape,y_sm.shape)

(1312, 22) (1312, 56)


In [ ]:
hist = model.fit(X_sm, y_sm, batch_size=batch_size, epochs=num_epochs, callbacks=callbacks_list, validation_split=0.1, shuffle=True, verbose=2)

Train on 1180 samples, validate on 132 samples
Epoch 1/8
 - 1s - loss: 0.1184 - accuracy: 0.9762 - val_loss: 0.0999 - val_accuracy: 0.9821
Epoch 2/8
 - 1s - loss: 0.1082 - accuracy: 0.9794 - val_loss: 0.0993 - val_accuracy: 0.9821
Epoch 3/8
 - 1s - loss: 0.0992 - accuracy: 0.9807 - val_loss: 0.0979 - val_accuracy: 0.9821
Epoch 4/8
 - 1s - loss: 0.0943 - accuracy: 0.9816 - val_loss: 0.0959 - val_accuracy: 0.9821
Epoch 5/8
 - 1s - loss: 0.0893 - accuracy: 0.9818 - val_loss: 0.0959 - val_accuracy: 0.9823
Epoch 00005: early stopping


In [ ]:
print(model.test_on_batch(val_X,val_Y))
model.metrics_names

[0.07347066, 0.98222303]


['loss', 'accuracy']

In [ ]:
from sklearn.utils import class_weight
y_train_labels = np.argmax(train_Y, axis =1)
class_weight = class_weight.compute_class_weight('balanced',np.unique(y_train_labels),y_train_labels)

In [ ]:
hist = model.fit(X_sm, y_sm, batch_size=batch_size, epochs=num_epochs, class_weight=class_weight, callbacks=callbacks_list, validation_split=0.1, shuffle=True, verbose=2)

Train on 1180 samples, validate on 132 samples
Epoch 1/8
 - 1s - loss: 0.0861 - accuracy: 0.9821 - val_loss: 0.0947 - val_accuracy: 0.9823
Epoch 2/8
 - 1s - loss: 0.0819 - accuracy: 0.9827 - val_loss: 0.0930 - val_accuracy: 0.9823
Epoch 3/8
 - 1s - loss: 0.0791 - accuracy: 0.9829 - val_loss: 0.0929 - val_accuracy: 0.9823
Epoch 4/8
 - 1s - loss: 0.0767 - accuracy: 0.9830 - val_loss: 0.0934 - val_accuracy: 0.9823
Epoch 5/8
 - 1s - loss: 0.0737 - accuracy: 0.9834 - val_loss: 0.0897 - val_accuracy: 0.9824
Epoch 00005: early stopping


In [ ]:
print(model.test_on_batch(val_X,val_Y))
model.metrics_names

[0.061612327, 0.98298377]


['loss', 'accuracy']

In [ ]:
hist = model.fit(train_X, train_Y, batch_size=batch_size, epochs=num_epochs, class_weight=class_weight, callbacks=callbacks_list, validation_split=0.1, shuffle=True, verbose=2)

Train on 1127 samples, validate on 126 samples
Epoch 1/8
 - 1s - loss: 0.0733 - accuracy: 0.9834 - val_loss: 0.0632 - val_accuracy: 0.9834
Epoch 2/8
 - 1s - loss: 0.0694 - accuracy: 0.9836 - val_loss: 0.0617 - val_accuracy: 0.9837
Epoch 3/8
 - 1s - loss: 0.0675 - accuracy: 0.9842 - val_loss: 0.0599 - val_accuracy: 0.9837
Epoch 4/8
 - 1s - loss: 0.0660 - accuracy: 0.9837 - val_loss: 0.0586 - val_accuracy: 0.9837
Epoch 5/8
 - 1s - loss: 0.0633 - accuracy: 0.9845 - val_loss: 0.0558 - val_accuracy: 0.9844
Epoch 00005: early stopping


In [ ]:
print(model.test_on_batch(val_X,val_Y))
model.metrics_names

[0.05251307, 0.9852274]


['loss', 'accuracy']

In [ ]:
model.evaluate(val_X,val_Y)

314/314 [==============================] - 0s 1ms/step


[0.05251307095027274, 0.9855550527572632]